In [ ]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt

# from folium import LayerControl
from eo_tools_dev.util import show_insar_coh, show_insar_phi, show_sar_int
import geopandas as gpd
from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

# Set up parameters and output dir

## Important
- The products must be **unzipped** either manually or by using `extract=True` in the download call. 
- In the following products, only IW1 and IW2 are relevant as IW3 contains only water pixels.

In [ ]:
# change to your custom locations
data_dir = "/data/S1"

ids = [
 "S1A_IW_SLC__1SDV_20230904T063730_20230904T063757_050174_0609E3_DAA1", 
 "S1A_IW_SLC__1SDV_20230916T063730_20230916T063757_050349_060FCD_6814"
]
primary_dir = f"{data_dir}/{ids[0]}.SAFE"
secondary_dir = f"{data_dir}/{ids[1]}.SAFE"
outputs_prefix="/data/res/test-full-processor"

# Download S-1 products

In [ ]:
# load a geometry
file_aoi = "../data/Morocco_tiny.geojson"
shp = gpd.read_file(file_aoi).geometry[0]

search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2023-09-03",
    "end": "2023-09-17",
    "geom": shp
}

results, _ = dag.search(**search_criteria)
to_dl = [it for it in results if it.properties["id"] in ids]
print(f"{len(to_dl)} products to download")
dag.download_all(to_dl, outputs_prefix="/data/S1/", extract=False)

# Pre-process InSAR pair

In [ ]:
from eo_tools.S1.process import process_insar

out_dir = process_insar(
    dir_prm=primary_dir,
    dir_sec=secondary_dir,
    outputs_prefix=outputs_prefix,
    aoi_name=None,
    shp=shp,
    pol="vh",
    write_coherence=True,
    write_interferogram=True,
    write_primary_amplitude=True,
    write_secondary_amplitude=True,
    apply_fast_esd=True,
    subswaths=["IW1", "IW2", "IW3"],
    dem_upsampling=1.8,
    dem_force_download=False,
    dem_buffer_arc_sec=40,
    boxcar_coherence=[3, 10],
    multilook=[1, 4],
    warp_kernel="bicubic",
    warp_kernel_phase="nearest",
    clip_to_shape=False
)

if not set, change the path in the next cell to:
`"/data/res/test-new-processor/S1_InSAR_2023-09-04-063730__2023-09-16-063730/phi_vh.tif"`

In [ ]:
show_insar_coh("/data/res/test-full-processor/S1_InSAR_2023-09-04-063730__2023-09-16-063730/coh_vh.tif")

In [ ]:
show_insar_phi("/data/res/test-full-processor/S1_InSAR_2023-09-04-063730__2023-09-16-063730/phi_vh.tif")

In [ ]:
show_sar_int("/data/res/test-full-processor/S1_InSAR_2023-09-04-063730__2023-09-16-063730/amp_prm_vh.tif")

In [ ]:
toto = "/data/res/test-full-processor/S1_InSAR_2023-09-04-063730__2023-09-16-063730/sar/coh_vh_iw1_geo.tif"
import rasterio
import rasterio
with rasterio.open(toto) as ds:
    # arr = ds.read(1)
    print(ds.profile)
    